## Zadanie domowe: BBHE i DSIHE

W klasycznym wyrównywaniu histogramu HE  po wykonaniu operacji jasność obrazu ulega zmianie.
Dało się to zaobserwować podczas przeprowadzonych eksperymentów.
Jeśli nie to należy uruchomić skrypt z sekcji A i zwrócić na to uwagę.
Średnia jasność dąży do środkowego poziomu szarości.
Jest to wada i dlatego klasyczne HE ma ograniczone zastosowanie.

Powstało sporo metod, które eliminują to niekorzystne zjawisko.
Najprostsze z nich polegają na dekompozycji obrazu wejściowego na dwa podobrazy (wg. pewnego kryterium).
Następnie operacja HE wykonywana jest dla tych podobrazów.

Dwie znane z literatury metody to:
- Bi-Histogram Equalization
- DSIHE - Dualistic Sub-Image Histogram Equalization

W metodzie BBHE za kryterium podziału przyjmuje się średnią jasność w obrazie.
W DSIHE obraz dzieli się na dwa podobrazy o takiej samej liczbie pikseli (jaśniejszych i ciemniejszych).

W ramach zadania należy zaimplementować wybraną metodę: BBHE lub DSIHE (ew. obie).

1. Wczytaj obraz *jet.bmp* i wylicz jego histogram.
2. W kolejnym kroku należy wyznaczyć próg podziału obrazu na dwa podobrazy (*lm*).
3. Dla BBHE wyznacz średnią jasność obrazu. Dla DSIHE można wykorzystać histogram skumulowany.
Należy znaleźć poziom jasności który znajduje się "w połowie" histogramu skumulowanego.
W tym celu warto stworzyć tablicę, zawierającą moduł histogramu skumulowanego pomniejszonego o połowę liczby pikseli.
Następnie znaleźć minimum - `np.argmin`.
4. Dalej należy podzielić histogram oryginalnego obrazu na dwa histogramy *H1* i *H2*.
Dla każdego z nich wyliczyć histogram skumulowany ($C_1$ i $C_2$) i wykonać normalizację.
Normalizacja polega na podzieleniu każdego histogramu przez jego największy element.
5. Na podstawie histogramów skumulowanych należy stworzyć przekształcenie LUT.
Należy tak przeskalować $C_1$ i $C_2$, aby uzyskać jednorodne przekształcenie.
Tablicę $C_1$ wystarczy pomnożyć przez próg podziału.
Tablicę $C_2$ należy przeskalować do przedziału: $<lm+1; 255>$, gdzie $lm$ jest progiem podziału.<br>
$C_{1n} = (lm)*C1;$<br>
$C_{2n} = lm+1 + (255-lm-1)*C2;$<br>
Następnie dwie części tablicy przekodowań należy połączyć.
6. Na koniec należy wykonać operację LUT i wyświetlić wynik wyrównywania histogramu.
Porównaj wynik operacji BBHE lub DSIHE z klasycznym HE.

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/'

fileName = 'jet.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

In [ ]:
photo = cv2.imread("jet.bmp", cv2.IMREAD_GRAYSCALE)
histogram = cv2.calcHist([photo], [0], None, [256], [0, 255])

def transform(lm, photo):
    h1 = histogram.copy()
    h2 = histogram.copy()
    h1[int(lm):] = 0
    h2[:int(lm)] = 0
    hn1 = h1.cumsum()
    hn1 /= max(hn1)
    hn1 *= lm
    hn2 = h2.cumsum()
    hn2 /= max(hn2)
    hn2 = lm + 1 + (255 - lm - 1) * hn2
    mask = np.concatenate([hn1[:int(lm)], hn2[int(lm):]]).astype('uint8')
    photo = cv2.LUT(photo, mask)
    return photo

lm_bbhe = np.mean(photo)
bbhe_photo = transform(lm_bbhe, photo)

skumulowany = histogram.cumsum()
num = skumulowany[-1]/2
lm_dsihe = np.argmin(np.abs(skumulowany - num))
dsihe_photo = transform(lm_dsihe, photo)

equalized_photo = cv2.equalizeHist(photo)

plt.Figure(figsize=(1, 1))
plt.plot(histogram, label="Histogram")
plt.plot(skumulowany, label="Skumulowany")
plt.grid()
plt.legend()
plt.show()

fig, axes = plt.subplots(3, 2)
fig.set_size_inches(20, 20)
titles = ["Wyrównanie bbhe", "Wyrównanie dsihe", "Wyrównanie HE"]

for num, img in enumerate([bbhe_photo, dsihe_photo, equalized_photo]):
    axes[num, 0].imshow(img, 'gray', vmin=0, vmax=255)
    axes[num, 0].set_title(titles[num])
    histogram = cv2.calcHist([img], [0], None, [256], [0, 255])
    axes[num, 1].plot(histogram, label="Histogram", color='green')
    cumsum = histogram.cumsum()
    cumsum = cumsum/max(cumsum) * max(histogram)
    axes[num, 1].plot(cumsum)

plt.show()